# Custom Flow-gen for Training dataset -- Model creation - Botnet Detection

In [1]:
import graphlab as gl
import tensorflow as tf
import matplotlib.pyplot as plt

This non-commercial license of GraphLab Create for academic use is assigned to aparna.mahalingam@sjsu.edu and will expire on April 06, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1525929112.log


In [2]:
import os
import sys, re

os.getcwd()

'/home/aparna/Desktop/botnet_detection'

In [3]:
# Training dataset used: ISCX_Botnet-Training.csv (after extracting all the raw features from Wireshark)
SF_tr = gl.SFrame.read_csv('ISCX_Botnet-Training.csv')
print SF_tr.head

Finished parsing file /home/aparna/Desktop/botnet_detection/ISCX_Botnet-Training.csv

Parsing completed. Parsed 100 lines in 8.46163 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,float,str,str,int,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 207809 lines. Lines per second: 26992.1

Read 622854 lines. Lines per second: 39411.6

...

Read 8771208 lines. Lines per second: 50271.2

Read 9168021 lines. Lines per second: 50289.4

Finished parsing file /home/aparna/Desktop/botnet_detection/ISCX_Botnet-Training.csv

Parsing completed. Parsed 9388270 lines in 184.567 secs.

<bound method SFrame.head of Columns:
	No.	int
	Time	float
	Source	str
	Protocol	str
	Length	int
	Source Port	str
	Destination	str
	Destination Port	str
	udp_Length	str
	Protocols in frame	str
	Info	str
	Differentiated Services Field	str
	Sequence number	str
	Next sequence number	str
	IP_Flags	str
	tcp_Flags	str
	Answer RRs	str
	Source GeoIP Country	str
	Source GeoIP Latitude	str
	Source GeoIP Longitude	str
	Destination GeoIP Country	str
	Destination GeoIP Latitude	str
	Destination GeoIP Longitude	str
	TCP Segment Len	str
	Time to live	str
	New Column	int

Rows: 9388270

Data:
+-----+------------+-------------------+----------+--------+-------------+
| No. |    Time    |       Source      | Protocol | Length | Source Port |
+-----+------------+-------------------+----------+--------+-------------+
|  1  |    0.0     |   Cisco_db:19:c3  |   ARP    |   60   |             |
|  2  |  8.982709  |   Cisco_db:19:c3  |   ARP    |   60   |             |
|  3  | 50.099564  |   Cisco_db:19:c3  | 

In [4]:
SF_tr = SF_tr[ (SF_tr['Source Port'] != '') & (SF_tr['Destination Port'] != '') ]
# removing all empty cells from the src-dst port columns

In [5]:
# converting to base-16 integers (hex values)
SF_tr['tcp_Flags'] = SF_tr['tcp_Flags'].apply(lambda x:int(x, 16) if x != '' else 0)

In [6]:
Protocols = sorted(SF_tr['Protocol'].unique())
print Protocols

['ANSI C12.22', 'ASAP', 'ATH', 'ATMTCP', 'AX4000', 'BAT_VIS', 'BROWSER', 'BitTorrent', 'CAT-TP', 'CLASSIC-STUN', 'DCERPC', 'DHCP', 'DHCPv6', 'DIAMETER', 'DNS', 'ECHO', 'ECMP', 'EGD', 'ESP', 'FF', 'FTP', 'FTP-DATA', 'FTP-DATA/XML', 'GPRS-NS', 'GTP', 'Gnutella', 'H.225.0', 'H1', 'HTTP', 'HTTP/XML', 'ICMP', 'IEEE 802.15.4', 'IMAP', 'IPv6', 'IRC', 'LANMAN', 'LISP', 'LLC', 'LLMNR', 'Laplink', 'MDNS', 'MIH', 'MQTT', 'MSMMS', 'MSNMS', 'MiNT', 'NBNS', 'NBSS', 'NDPS', 'NTP', 'Omni-Path', 'PCEP', 'PKIX-CRL', 'PKTC', 'POP', 'Pathport', 'QUAKE', 'QUAKE3', 'QUAKEWORLD', 'RMI', 'RSIP', 'RTCP', 'RTMP', 'SIGCOMP', 'SIP', 'SMB', 'SMTP', 'SMTP|IMF', 'SNMP', 'SSDP', 'SSH', 'SSHv2', 'SSL', 'SSLv2', 'SSLv3', 'Socks', 'TCP', 'TCPCL', 'TDS', 'TLSv1', 'UDP', 'VICP', 'VNC', 'WSP', 'X11', 'eDonkey']


In [6]:
# more malicious IPs specific to malware have been added to the list provided by the Canadian Institute for Cybersecurity 

ip_list = [
    "94.250.250.18",
    "94.102.49.125",
    "161.202.132.62",
    "54.167.101.139",
    "147.32.84.180",
    "147.32.84.170",
    "147.32.84.150",
    "147.32.84.140",
    "147.32.84.130",
    "147.32.84.160",
    "10.0.2.15",
    "192.168.106.141",
    "192.168.106.131",
    "172.16.253.130",
    "172.16.253.131",
    "172.16.253.129",
    "172.16.253.240",
    "74.78.117.238",
    "158.65.110.24",
    "192.168.3.35",
    "192.168.3.25",
    "192.168.3.65",
    "172.29.0.116",
    "172.29.0.109",
    "172.16.253.132",
    "192.168.248.165",
    "10.37.130.4",
    "131.202.243.84", 
    "198.164.30.2", "192.168.5.122", 
    "192.168.5.122", 
    "192.168.5.122", "192.168.5.122", "192.168.5.122", 
    "192.168.2.110", 
    "192.168.4.120", "192.168.1.103",
    "192.168.2.113", "192.168.4.118" , "192.168.2.109", "192.168.2.105", "192.168.5.122"]

MasterBots = [ ("192.168.2.112", "131.202.243.84"), ("192.168.5.122", "198.164.30.2"), ("192.168.2.110", "192.168.5.122"),( "192.168.4.118", "192.168.5.122"), ("192.168.2.113", "192.168.5.122"), ("192.168.1.103", "192.168.5.122"), ("192.168.4.120", "192.168.5.122"), ("192.168.2.112", "192.168.2.110"), ("192.168.2.112", "192.168.4.120"), ("192.168.2.112", "192.168.1.103"), ("192.168.2.112", "192.168.2.113"), ("192.168.2.112", "192.168.4.118"), ("192.168.2.112", "192.168.2.109"), ("192.168.2.112", "192.168.2.105"), ("192.168.1.105", "192.168.5.122")]

ip_list = gl.SArray(ip_list)
ip_list = ip_list.unique() 
ip_list = list(ip_list)

def fcheckIP(x):
    if (x['Source'] in ip_list) or (x['Destination'] in ip_list):
        return 1
    else:
        if ((x['Source'],x['Destination'])  in MasterBots ) or ((x['Destination'],x['Source'])  in MasterBots ) :
            return 1
        else:
            return 0
        

SF_tr['isBot'] = SF_tr[['Source','Destination']].apply(lambda x: fcheckIP(x))

In [7]:
# Packets labelled as botnet -- % of these packets present
botlabels = len(SF_tr[SF_tr['isBot'] == 1])
botpercent = ((botlabels * 1.0) / len(SF_tr))* 100
print botlabels

3874036


In [8]:
print('{:4.3f}% botnet labelled packets present in the training set'.format(botpercent))

41.387% botnet labelled packets present in the training set


## Flows are sorted into bi-directional (for efficient and effective flow differentiation / identification) and given unique Flow IDs:

In [9]:
def flow_id(x):
    # each packet is sorted according to its source and destination IPs
    if x['Source'] > x['Destination']:
        return( x['Source'] + '-' + x['Destination'] + '-' + str(x['Source Port']) + '-' + str(x['Destination Port'])
    + '-' + x['Protocol'] )
    else:
        return( x['Destination'] + '-' + x['Source'] + '-' + str(x['Destination Port']) + '-' + str(x['Source Port'])
               + '-' + x['Protocol'] )
    
SF_tr['UFid'] = SF_tr.apply(lambda x:flow_id(x))

In [10]:
# make sure incoming flow & outgoing flow are separately labeled for identifying IOPR -- Incoming-Outgoing Packet Ratio
SF_tr['Forward'] = SF_tr.apply( lambda x:1 if x['Source'] > x['Destination'] else 0 )

# comparing two different flows based on columns
def compareUF(x, y):
    if x != y:
        return False
    return True

In [11]:
import pickle 

In [12]:
# Code for logic of Bidirectional flow identification
FlowNo = 0 # Unique Flow Number for each flow
prev = None
Flow = []     # Stores all flows in form of list of dictionaries 

count = 0
fc = 0
startTime = None   # Start Time of each flow to implement timeout

SF_tr = SF_tr.sort(['UFid','Time'])
print 'Done Sorting'
for x in SF_tr:
    if count % 500000 == 0:
        print 'Running ' + str(count) + ' Done !'  # keeping track of the dataset parsing
    count = count+1
    
    if prev is None:
        if startTime is None:
            startTime = x['Time']
        Flow.append(FlowNo)  
        prev = x['UFid']
        
    elif compareUF(x['UFid'], prev):
        if x['tcp_Flags'] & 1:
            Flow.append(FlowNo)
            prev = None
            startTime = None
            FlowNo = FlowNo + 1

        elif (x['Time'] - startTime) >= 3600:
            FlowNo = FlowNo + 1
            Flow.append(FlowNo)
            prev = None
            startTime = x['Time']
            
        else:    
            Flow.append(FlowNo)
            prev = x['UFid']

    else:
        FlowNo = FlowNo + 1
        Flow.append(FlowNo)
        prev = x['UFid']
        startTime = x['Time']


print len(gl.SArray(Flow).unique())


SF_tr['Flow'] = gl.SArray(Flow)
temp = SF_tr.groupby('Flow',{ 'Count':gl.aggregate.COUNT() })

print len(temp[temp['Count'] > 1])

Done Sorting
Running 0 Done !
Running 500000 Done !
Running 1000000 Done !
Running 1500000 Done !
Running 2000000 Done !
Running 2500000 Done !
Running 3000000 Done !
Running 3500000 Done !
Running 4000000 Done !
Running 4500000 Done !
Running 5000000 Done !
Running 5500000 Done !
Running 6000000 Done !
Running 6500000 Done !
Running 7000000 Done !
Running 7500000 Done !
Running 8000000 Done !
Running 8500000 Done !
Running 9000000 Done !
485622
402707


In [13]:
FlowNo = 0 # Unique Flow Number for each flow
prev = None
Flow = []     ##Stores all flows in form of list of dictionary 

count = 0
fc = 0
startTime = None   ##Start Time of each flow to implement timeout

print 'Done Sorting'

for x in SF_tr:
    if count % 500000 == 0:
        print 'Running ' + str(count) + ' Done !'
        
    count += 1
    
    if prev is None:
        if startTime is None:
            startTime = x['Time']
        Flow.append(FlowNo)
        
        prev = x['UFid']
        
    elif compareUF(x['UFid'],prev):
        if x['tcp_Flags'] & 1 or x['tcp_Flags'] & 4:  # difference in implementation
            Flow.append(FlowNo)
            prev = None
            startTime = None
            FlowNo = FlowNo + 1

        elif x['Time']-startTime>=3600:
            FlowNo = FlowNo + 1
            Flow.append(FlowNo)
            prev = None
            startTime = x['Time']
            
        else:
            
            Flow.append(FlowNo)
            prev = x['UFid']

    else:
        FlowNo = FlowNo + 1
        Flow.append(FlowNo)
        prev = x['UFid']
        startTime = x['Time']


print len(gl.SArray(Flow).unique())


SF_tr['Flow'] = gl.SArray(Flow)

temp = SF_tr.groupby('Flow',{ 'Count':gl.aggregate.COUNT() })

print len(temp[temp['Count'] > 1])

Done Sorting
Running 0 Done !
Running 500000 Done !
Running 1000000 Done !
Running 1500000 Done !
Running 2000000 Done !
Running 2500000 Done !
Running 3000000 Done !
Running 3500000 Done !
Running 4000000 Done !
Running 4500000 Done !
Running 5000000 Done !
Running 5500000 Done !
Running 6000000 Done !
Running 6500000 Done !
Running 7000000 Done !
Running 7500000 Done !
Running 8000000 Done !
Running 8500000 Done !
Running 9000000 Done !
500983
418651


In [14]:
# sorting the flows yet again to ensure proper bi-directional flow sorting 

FlowNo = 0 # unique Flow Number for each flow
prev = None
Flow = []     # stores all flows in the form of a list of dictionaries

count = 0
fc = 0
startTime = None   # Start-Time of each flow recorded to implement timeout

SF_tr = SF_tr.sort(['UFid','Time'])
print 'Done Sorting'

for x in SF_tr:
    if count % 500000 == 0:
        print 'Running ' + str(count) + ' Done !'
        
    count += 1
    
    if prev is None:
        if startTime is None:
            startTime = x['Time']
        Flow.append(FlowNo)
        
        prev = x['UFid']
    elif compareUF(x['UFid'],prev):
        if x['tcp_Flags'] & 1: # previous round of sorting did not involve sorting again by unique flow-ids or comparing 'tcp_flags' with 4
            Flow.append(FlowNo)
            prev = None
            startTime = None
            FlowNo = FlowNo + 1

        elif (x['Time'] - startTime) >=3600:
            FlowNo = FlowNo + 1
            Flow.append(FlowNo)
            prev = None
            startTime = x['Time']
            
        else:
            
            Flow.append(FlowNo)
            prev = x['UFid']

    else:
        FlowNo = FlowNo + 1
        Flow.append(FlowNo)
        prev = x['UFid']
        startTime = x['Time']


print len(gl.SArray(Flow).unique())

Done Sorting
Running 0 Done !
Running 500000 Done !
Running 1000000 Done !
Running 1500000 Done !
Running 2000000 Done !
Running 2500000 Done !
Running 3000000 Done !
Running 3500000 Done !
Running 4000000 Done !
Running 4500000 Done !
Running 5000000 Done !
Running 5500000 Done !
Running 6000000 Done !
Running 6500000 Done !
Running 7000000 Done !
Running 7500000 Done !
Running 8000000 Done !
Running 8500000 Done !
Running 9000000 Done !
485622


In [15]:
SF_tr['Flow'] = gl.SArray(Flow)

temp = SF_tr.groupby('Flow',{ 'Count':gl.aggregate.COUNT() }) # to determine how many flows are present in the data / traffic

print len(temp[temp['Count'] > 1])

402707


In [16]:
temp = SF_tr.groupby('Flow',{   'NumBots' : gl.aggregate.SUM('isBot')    }) # categorizing based on the isBot criterion

NumBotFlows = len(temp[temp['NumBots'] > 1])
print('Number of bot-flows: {}'.format(NumBotFlows))

botflowspercent = (NumBotFlows *1.0) / len(SF_tr['Flow'].unique())
print('{:2.4f}% of bot-flows are seen among all sorted flows'.format(botflowspercent * 100))

Number of bot-flows: 235640
48.5233% of bot-flows are seen among all sorted flows


In [17]:
len(Flow) # out of the total 9388270 total packets

9360525

In [18]:
len(SF_tr)

9360525

In [19]:
SF_tr['FlowNo.'] = gl.SArray(Flow)

In [20]:
# checking to see if there are any discrepancies in the data --
# i.e., if flows are in decreasing order, contrary to the conditions applied for sorting flows

Flow[:45]

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 8,
 9,
 9,
 10,
 10,
 10,
 10,
 11]

In [21]:
pickle.dump(Flow,open('Flow.pkl','w'))

In [22]:
SF_tr.remove_column("New Column")

No.,Time,Source,Protocol,Length,Source Port,Destination,Destination Port,udp_Length
2255379,-121112382.05,,IEEE 802.15.4,70,49999,,30319,36
2145566,-121113062.71,,IEEE 802.15.4,88,30319,,49999,50
2145702,-121113061.371,,IEEE 802.15.4,84,30319,,49999,50
2152740,-121113006.007,,IEEE 802.15.4,96,30319,,49999,73
2153680,-121113001.2,,IEEE 802.15.4,96,30319,,49999,71
2170704,-121112850.576,,IEEE 802.15.4,96,30319,,49999,71
2180568,-121112781.031,,IEEE 802.15.4,96,30319,,49999,71
2196536,-121112679.898,,IEEE 802.15.4,96,30319,,49999,71
2212282,-121112603.31,,IEEE 802.15.4,96,30319,,49999,71
2255350,-121112382.224,,IEEE 802.15.4,96,30319,,49999,108


In [23]:
SF_tr.save('ISCX_Botnet-Testing_Ports_Only_Sorted_Flow_BD.csv')

In [24]:
# this checks if the frame.protocols (Protocols in frame) field contains 'udp'
def frameproto_udp(x):
    if 'udp' in x.split(':'):
        return 1
    return 0
SF_tr['hasUDP'] = SF_tr['Protocols in frame'].apply(lambda x:frameproto_udp(x))


In [25]:
# this checks if the frame.protocols (Protocols in frame) field contains 'smb' -- useful in detecting and server message block and file sharing access related packets are being exchanged
def frameproto_smb(x):
    if 'smb' in x.split(':'):
        return 1
    return 0
SF_tr['hasSMB'] = SF_tr['Protocols in frame'].apply(lambda x:frameproto_smb(x))


In [26]:
SF_tr.head

<bound method SFrame.head of Columns:
	No.	int
	Time	float
	Source	str
	Protocol	str
	Length	int
	Source Port	str
	Destination	str
	Destination Port	str
	udp_Length	str
	Protocols in frame	str
	Info	str
	Differentiated Services Field	str
	Sequence number	str
	Next sequence number	str
	IP_Flags	str
	tcp_Flags	int
	Answer RRs	str
	Source GeoIP Country	str
	Source GeoIP Latitude	str
	Source GeoIP Longitude	str
	Destination GeoIP Country	str
	Destination GeoIP Latitude	str
	Destination GeoIP Longitude	str
	TCP Segment Len	str
	Time to live	str
	isBot	int
	UFid	str
	Forward	int
	Flow	int
	FlowNo.	int
	hasUDP	int
	hasSMB	int

Rows: 9360525

Data:
+---------+----------------+--------+---------------+--------+-------------+-------------+
|   No.   |      Time      | Source |    Protocol   | Length | Source Port | Destination |
+---------+----------------+--------+---------------+--------+-------------+-------------+
| 2255379 | -121112382.05  |        | IEEE 802.15.4 |   70   |    49999    |  

In [27]:
# IOPR -- Ratio of incoming to outgoing packets (very important in terms of detecting any suspicious IRC type flows)
temp = SF_tr.groupby('FlowNo.',{
        'NumForward' : gl.aggregate.SUM('Forward'),
        'Total' : gl.aggregate.COUNT()
    })
temp['IOPR'] = temp.apply(lambda x:((x['Total']-x['NumForward'])*1.0)/x['NumForward'] if x['NumForward'] != 0 else (-1))
temp = temp['FlowNo.', 'IOPR']

In [28]:
# check how many flows are incoming
len(temp[temp['IOPR'] != -1])

360214

In [29]:
SF_tr = SF_tr.join(temp, on='FlowNo.') # add the IOPR column to SFrame
SF_tr['IOPR']

dtype: float
Rows: 9360525
[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ... ]

In [30]:
# Extraction of numerical and time-based derived packet features from raw features and sorted bi-directional flow

# First Packet Length

FlowFeatures = ['Source', 'Destination', 'Source Port', 'Destination Port', 'Protocol'] # 5-tuple flow-id features

FPL = SF_tr.groupby(['FlowNo.'],{
        'Time':gl.aggregate.MIN('Time')
    })
print len(FPL)
FPL = FPL.join(SF_tr,on =['FlowNo.','Time'])[['FlowNo.','Length']].unique()
FPL = FPL.groupby(['FlowNo.'],{
        'FPL':gl.aggregate.AVG('Length')
    })
print len(FPL)


SF_tr = SF_tr.join(FPL, on='FlowNo.')
del(FPL)

# Number of packets per flow
temp = SF_tr.groupby(['FlowNo.'],{
        'NumPackets':gl.aggregate.COUNT()
    })
print temp.head(3)
SF_tr = SF_tr.join(temp, on ='FlowNo.')
del(temp)

# Number of bytes exchanged
temp = SF_tr.groupby(['FlowNo.'],{
        'BytesEx':gl.aggregate.SUM('Length')
    })
SF_tr = SF_tr.join(temp, on ='FlowNo.')
del(temp)


# Standard deviation of packet length
temp = SF_tr.groupby(['FlowNo.'],{
        'StdDevLen':gl.aggregate.STDV('Length')
    })
SF_tr = SF_tr.join(temp, on ='FlowNo.')
del(temp)

# Same length packet ratio
temp2 = SF_tr.groupby(['FlowNo.'],{
        'SameLenPktRatio':gl.aggregate.COUNT_DISTINCT('Length')
    })

# temp from number of packets computation
temp = SF_tr.groupby(['FlowNo.'],{
        'NumPackets':gl.aggregate.COUNT()
    })
temp = temp.join(temp2,on='FlowNo.')
temp['SameLenPktRatio'] = temp['SameLenPktRatio']*1.0 / temp['NumPackets']
temp2 = None
temp = temp[['FlowNo.','SameLenPktRatio']]
SF_tr = SF_tr.join(temp, on ='FlowNo.')


# Duration of flow
timeF = SF_tr.groupby(['FlowNo.'],{
        'startTime':gl.aggregate.MIN('Time'),
        'endTime':gl.aggregate.MAX('Time')
    })
timeF['Duration'] = timeF['endTime'] - timeF['startTime']
timeF = timeF[['FlowNo.','Duration']]
SF_tr = SF_tr.join(timeF, on ='FlowNo.')


sorted(SF_tr.column_names())

485622
485622
+---------+------------+
| FlowNo. | NumPackets |
+---------+------------+
|  60114  |     2      |
|  196583 |     5      |
|  23107  |     1      |
+---------+------------+
[3 rows x 2 columns]



['Answer RRs',
 'BytesEx',
 'Destination',
 'Destination GeoIP Country',
 'Destination GeoIP Latitude',
 'Destination GeoIP Longitude',
 'Destination Port',
 'Differentiated Services Field',
 'Duration',
 'FPL',
 'Flow',
 'FlowNo.',
 'Forward',
 'IOPR',
 'IP_Flags',
 'Info',
 'Length',
 'Next sequence number',
 'No.',
 'NumPackets',
 'Protocol',
 'Protocols in frame',
 'SameLenPktRatio',
 'Sequence number',
 'Source',
 'Source GeoIP Country',
 'Source GeoIP Latitude',
 'Source GeoIP Longitude',
 'Source Port',
 'StdDevLen',
 'TCP Segment Len',
 'Time',
 'Time to live',
 'UFid',
 'hasSMB',
 'hasUDP',
 'isBot',
 'tcp_Flags',
 'udp_Length']

In [47]:
SF_tr.head

<bound method SFrame.head of Columns:
	No.	int
	Time	float
	Source	str
	Destination	str
	Protocol	str
	Length	int
	Protocols in frame	str
	Source Port	str
	Destination Port	str
	TCP Segment Len	str
	Next sequence number	str
	tcp_Flags	int
	Differentiated Services Field	str
	IP_Flags	str
	Total Length	str
	Time to live	str
	Info	str
	Source GeoIP Country	str
	Source GeoIP Latitude	str
	Source GeoIP Longitude	str
	Destination GeoIP Latitude	str
	Destination GeoIP Longitude	str
	Destination GeoIP Country	str
	Answer RRs	str
	udp_Length	str
	Answer RRs.1	str
	Destination Host	str
	isBot	int
	UFid	str
	Forward	int
	Flow	int
	FlowNo.	int
	hasUDP	int
	hasSMB	int
	IOPR	float
	FPL	float
	NumPackets	int
	BytesEx	int
	StdDevLen	float
	SameLenPktRatio	float
	Duration	float

Rows: 8643056

Data:
+--------+---------------+---------------+-----------------+----------+--------+
|  No.   |      Time     |     Source    |   Destination   | Protocol | Length |
+--------+---------------+---------------+--

In [31]:
SF_tr.print_rows(num_rows=17, num_columns=25)

+---------+----------------+-------------------------+---------------+--------+
|   No.   |      Time      |          Source         |    Protocol   | Length |
+---------+----------------+-------------------------+---------------+--------+
| 2255379 | -121112382.05  |                         | IEEE 802.15.4 |   70   |
| 2145566 | -121113062.71  |                         | IEEE 802.15.4 |   88   |
| 2145702 | -121113061.371 |                         | IEEE 802.15.4 |   84   |
| 2152740 | -121113006.007 |                         | IEEE 802.15.4 |   96   |
| 2153680 |  -121113001.2  |                         | IEEE 802.15.4 |   96   |
| 2170704 | -121112850.576 |                         | IEEE 802.15.4 |   96   |
| 2180568 | -121112781.031 |                         | IEEE 802.15.4 |   96   |
| 2196536 | -121112679.898 |                         | IEEE 802.15.4 |   96   |
| 2212282 | -121112603.31  |                         | IEEE 802.15.4 |   96   |
| 2255350 | -121112382.224 |            

In [32]:
SF_tr.print_rows(num_rows=25, num_columns=17)

+---------+----------------+-------------------------+---------------+--------+
|   No.   |      Time      |          Source         |    Protocol   | Length |
+---------+----------------+-------------------------+---------------+--------+
| 2255379 | -121112382.05  |                         | IEEE 802.15.4 |   70   |
| 2145566 | -121113062.71  |                         | IEEE 802.15.4 |   88   |
| 2145702 | -121113061.371 |                         | IEEE 802.15.4 |   84   |
| 2152740 | -121113006.007 |                         | IEEE 802.15.4 |   96   |
| 2153680 |  -121113001.2  |                         | IEEE 802.15.4 |   96   |
| 2170704 | -121112850.576 |                         | IEEE 802.15.4 |   96   |
| 2180568 | -121112781.031 |                         | IEEE 802.15.4 |   96   |
| 2196536 | -121112679.898 |                         | IEEE 802.15.4 |   96   |
| 2212282 | -121112603.31  |                         | IEEE 802.15.4 |   96   |
| 2255350 | -121112382.224 |            

In [33]:
features = ['Answer RRs',
 'BytesEx',
 'Destination',
 'Destination Port',
 'Differentiated Services Field',
 'Duration',
 'FPL',
 'IP_Flags',
 'Info',
 'Length',
 'Next sequence number',
 'No.',
 'NumPackets',
 'Protocol',
 'Protocols in frame',
 'SameLenPktRatio',
 'Sequence number',
 'Source',
 'Source Port',
 'StdDevLen',
 'TCP Segment Len',
 'Time',
 'Time to live',
 'isBot',
 'tcp_Flags',
 'FlowNo.',
 'udp_Length',
 'IOPR']
SF_tr = SF_tr[features]

# Average packets per second
temp =  SF_tr.groupby(['FlowNo.'],{
        'NumPackets':gl.aggregate.COUNT()
    })
temp = temp.join(timeF,on=['FlowNo.'])
temp['AvgPktPerSec'] = temp.apply(lambda x:0.0 if x['Duration'] == 0.0 else x['NumPackets']*1.0/x['Duration'])
temp = temp[['FlowNo.','AvgPktPerSec']]
SF_tr = SF_tr.join(temp, on ='FlowNo.')


# Average Bits Per Second
temp = SF_tr.groupby(['FlowNo.'],{
        'BytesEx':gl.aggregate.SUM('Length')
    })
temp = temp.join(timeF,on=['FlowNo.'])
temp['BitsPerSec'] = temp.apply( lambda x:0.0 if x['Duration'] == 0.0 else (x['BytesEx'] * 8.0)/x['Duration'] )
temp = temp[['FlowNo.', 'BitsPerSec']]
SF_tr = SF_tr.join(temp, on ='FlowNo.')


# Average Packet Lentgth
temp = SF_tr.groupby(['FlowNo.'],{
        'APL':gl.aggregate.AVG('Length')
    })
SF_tr = SF_tr.join(temp, on ='FlowNo.')

# Number of Reconnects, sort FlowNo, SeqNo
def tfn(x):
    if 'udp' in x.split(':') or 'tcp' in x.split(':'): # 'smb' protocols can be considered 
        return 1
    return 0
temp = list(SF_tr['Protocols in frame'].apply(lambda x:tfn(x)))

print len(temp)

print sum(temp)


print('HTTP URL-encodings in frame.protocols: {}'.format(len(SF_tr[SF_tr['Protocols in frame'] == 'eth:ethertype:ip:icmp:ip:tcp:http:urlencoded-form'])))
print('TCP in frame.protocols: {}'.format(len(SF_tr[SF_tr['Protocols in frame'] == 'eth:ethertype:ip:icmp:ip:tcp'])))
print('HTTP data in frame.protocols: {}'.format(len(SF_tr[SF_tr['Protocols in frame'] == 'eth:ethertype:ip:icmp:ip:tcp:http:data'])))

# Inter-arrival time
SF_tr['IAT'] = 0
SF_tr = SF_tr.sort(['FlowNo.','Time'])
prev = None
prevT = None
li = []
for x in SF_tr:
    if prev is None or x['FlowNo.'] != prev:
        li.append(0)
    else:
        li.append(x['Time'] - prevT)        
    prev = x['FlowNo.']
    prevT = x['Time']
SF_tr['IAT'] = gl.SArray(li)

print len(SF_tr)


print len(SF_tr[SF_tr['Protocols in frame'] == 'eth:ipx']) # internet packet exchange

9360525
9360525
HTTP URL-encodings in frame.protocols: 0
TCP in frame.protocols: 5398
HTTP data in frame.protocols: 0
9360525
0


In [36]:
# SF_tr.save('Bidirectional_Train_Bot_features_till_IAT.csv')

# Number of TCP Null packets

# Null Packets handling
def checkNull(x):
    if( x['TCP Segment Len'] == '0' or x['udp_Length'] == 8 ):
        return 1
    elif('ipx' in x['Protocols in frame'].split(':')):
        l = x['Length'] - 30
        if('eth' in x['Protocols in frame'].split(':')):
            l = l - 14
        if('ethtype' in x['Protocols in frame'].split(':')):
            l = l - 2
        if('llc' in x['Protocols in frame'].split(':')):
            l = l - 8
        if(l == 0 or l == -1):
            return 1
    return 0

SF_tr['isNull'] = SF_tr.apply(lambda x:checkNull(x))

NPEx = SF_tr.groupby(['FlowNo.'],{
        'NPEx':gl.aggregate.SUM('isNull')
    })
SF_tr = SF_tr.join(NPEx, on ='FlowNo.')


# Number of Reconnects - considering only TCP reconnects, using sequence number
recon = SF_tr[SF_tr['Sequence number'] != ''].groupby(['FlowNo.'],{
        'total_seq_no.' : gl.aggregate.COUNT('Sequence number'),
        'distinct_seq_no.' : gl.aggregate.COUNT_DISTINCT('Sequence number')
    })
recon['reconnects'] = recon['total_seq_no.'] - recon['distinct_seq_no.']
recon.head()
recon = recon[['FlowNo.','reconnects']]
SF_tr = SF_tr.join(recon, on='FlowNo.', how='left')

print len(SF_tr)

# To identify records where reconnect check was not applied, such as UDP, etc
SF_tr.fillna('reconnects',-1)

# A lot of these reconnects can be simple retransmissions - due to out of order/timeout etc
print min(SF_tr['Time']), max(SF_tr['Time'])

9360525
-121117184.726 780231.692992


In [37]:
features = SF_tr.column_names()

In [38]:
features

['Answer RRs',
 'BytesEx',
 'Destination',
 'Destination Port',
 'Differentiated Services Field',
 'Duration',
 'FPL',
 'IP_Flags',
 'Info',
 'Length',
 'Next sequence number',
 'No.',
 'NumPackets',
 'Protocol',
 'Protocols in frame',
 'SameLenPktRatio',
 'Sequence number',
 'Source',
 'Source Port',
 'StdDevLen',
 'TCP Segment Len',
 'Time',
 'Time to live',
 'isBot',
 'tcp_Flags',
 'FlowNo.',
 'udp_Length',
 'IOPR',
 'AvgPktPerSec',
 'BitsPerSec',
 'APL',
 'IAT',
 'isNull',
 'NPEx',
 'reconnects']

In [ ]:
# Bidirectional Flows combining all features and labels
SF_tr.save('bidirectional_train_botnet_all_features.csv')